In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

1) Workflow\
Para esse exercício, será necessário carregar discursos presidenciais feitos por Dilma e Temer em parte de seus mandatos:

In [2]:
link = 'https://github.com/FLS-6497/datasets/raw/main/aula5/discursos_presidenciais.csv'
discursos = pd.read_csv(link, sep=';')

,data,presidente,discurso,planalto
0,2012-12-16,Dilma,"Boa noite, povo de Fortaleza e povo do Ceará.\...",Outros locais
1,2015-10-09,Dilma,(Falha no áudio)... Luis Filipe de Carvalho Mo...,Outros locais
2,2013-06-14,Dilma,"Boa tarde, queridos trabalhadores, queridas tr...",Outros locais
3,2011-03-22,Dilma,Vocês viram o Omar ficando vermelho? Pois ele ...,Outros locais
4,2015-12-22,Dilma,Eu queria começar cumprimentando aqui os traba...,Outros locais
...,...,...,...,...
594,2017-05-16,Temer,"Olhem, prefeitos, prefeitas, vice-prefeitos, v...",Planalto
595,2017-11-29,Temer,Olha eu quero cumprimentar a todos.\nA pessoa ...,Outros locais
596,2017-12-18,Temer,"Eu quero cumprimentar a todos, começando pela ...",Outros locais
597,2017-12-12,Temer,Quero começar cumprimentando o João Martins da...,Outros locais


a) Pré-processamento\
Usando ferramentas de processamento de texto, implemente uma pequena pipeline para limpar dados e, quando estiver satisfeito com ela, crie uma função que aplique ela em uma nova lista textos.

In [19]:
vct = (
    CountVectorizer(
        stop_words=stopwords.words('portuguese'),
        token_pattern=r"(?u)\b\w\w\w+\b"
        )
        )
X = vct.fit_transform(discursos.discurso)   
print(X.shape)

(599, 31047)


b) Modelo\
Usando os frameworks de aprendizado de máquina, use a bag of words criada anteriormente para treinar algum modelo de classificação para predizer o nome do ou da presidente que proferiu cada discurso na amostra de teste.

In [11]:
# usar só transform na base de teste.

(599, 31473)


c) Validação\
Roda o item pedido em b) 100 vezes e salve os resultados de alguma métrica de validação.

2) Validação\
Como vimos, pré-processamento deve ser aplicado antes de fazermos split sample de validação (i.e., criar amostras de teste e de treino). Agora, implemente um workflow que leva isso em conta. Para tanto, você deverá criar uma função que separe textos em treino e teste, que aplique pré-processamento apenas na amostra de treino e que, depois, replique ele na amostra de teste para, então, rodar um algoritmo e calcular alguma métrica de validação.

3) Benchmark\
Usando as ferramentas que vimos, experimente com os seguintes pré-processamentos:
- Usando apenas palavras maiores do que 4 caracteres;
- Removendo palavras que não ocorrem em, pelo menos, 10 documentos;
- Removendo palavras que não ocorrem em, pelo menos, 10% dos documentos;
- Usando TF-IDF para normalizar os elementos da matriz bag of words;